In [56]:
import pandas as pd
import json
import pickle as p
import os

In [57]:
proj_path = 'test_5/'

In [58]:
with open(proj_path+'price_info.pkl', 'rb') as f:
    price_info = p.load(f)
    mean_prices = price_info[1]
    # Round to nearest 5 cent
    mean_prices = (mean_prices * 20).round() / 20
    mean_prices = pd.DataFrame(mean_prices).transpose()

mean_prices

,Price_1,Price_2,Price_11,Price_12,Price_21,Price_22,Price_31,Price_101,Price_102,Price_196,...,Price_7966,Price_7967,Price_16000,Price_16001,Price_16002,Price_16003,Price_16016,Price_16017,Price_16018,Price_16019
0,4.6,9.65,4.75,9.75,4.05,9.2,2.3,4.9,9.85,5.6,...,4.15,4.75,7.8,11.25,11.9,12.6,8.9,12.15,12.85,13.55


In [59]:
items = [int(x.split('_')[1]) for x in mean_prices.columns]

In [60]:
# Open item_models and save them to list
models_dict = {}

# Get Models List
for item in items:
    with open(proj_path + f'models/model_{item}.p', 'rb') as f:
        model = p.load(f)
    models_dict[item] = model

# Get Baseline Quantities
baseline_quantities_dict = {}

for item_id in models_dict.keys():
    item_model = models_dict[item_id]
    mean_prices = mean_prices[item_model._Booster.feature_names]
    pred = item_model.predict(mean_prices).round()
    baseline_quantities_dict[f'Qty_{item_id}'] = pred[0]
    
pd.DataFrame(baseline_quantities_dict, index=['baseline_qty']).transpose()

elasticity_dict = {}

# Get % Changes
for item in items:
    # If Prices go up by 10 cents...
    item_price_increase = mean_prices.copy()
    item_price_increase.loc[0,f'Price_{item}'] += 0.20
    # How much do quantities change?
    increase_quantities_dict = {}
    for item_j_id in items:
        item_j_model = models_dict[item_j_id]
        mean_prices = mean_prices[item_j_model._Booster.feature_names]
        pred = item_j_model.predict(item_price_increase).round()
        increase_quantities_dict[f'Qty_{item_j_id}'] = pred[0]
        
    # If Prices go up by 10 cents...
    item_price_decrease = mean_prices.copy()
    item_price_decrease.loc[0,f'Price_{item}'] -= 0.20
    # How much do quantities change?
    decrease_quantities_dict = {}
    for item_j_id in items:
        item_j_model = models_dict[item_j_id]
        mean_prices = mean_prices[item_j_model._Booster.feature_names]
        pred = item_j_model.predict(item_price_decrease).round()
        decrease_quantities_dict[f'Qty_{item_j_id}'] = pred[0]
        
    adjusted_quantities_df = pd.DataFrame([baseline_quantities_dict,
                                           increase_quantities_dict,
                                           decrease_quantities_dict
                                          ],
                                          index=['baseline_qty','increase_qty','decrease_qty']
                                         ).transpose()
#     print(adjusted_quantities_df)
    adjusted_quantities_df['increase_pct_chng'] = (adjusted_quantities_df['increase_qty'] - adjusted_quantities_df['baseline_qty']) / adjusted_quantities_df['baseline_qty']
    adjusted_quantities_df['decrease_pct_chng'] = (adjusted_quantities_df['decrease_qty'] - adjusted_quantities_df['baseline_qty']) / adjusted_quantities_df['baseline_qty']
    
    elasticity_dict[item] = adjusted_quantities_df.to_dict()
    
    

In [55]:
pd.DataFrame(elasticity_dict[3743])[]

,baseline_qty,increase_qty,decrease_qty,increase_pct_chng,decrease_pct_chng
Qty_202,2097.0,2477.0,1979.0,0.181211,-0.056271
Qty_1195,125.0,132.0,127.0,0.056000,0.016000
Qty_1230,9.0,9.0,8.0,0.000000,-0.111111
Qty_2531,256.0,304.0,235.0,0.187500,-0.082031
Qty_2589,128.0,145.0,154.0,0.132812,0.203125
Qty_3726,61.0,69.0,51.0,0.131148,-0.163934
Qty_3732,121.0,131.0,115.0,0.082645,-0.049587
Qty_3743,122.0,146.0,119.0,0.196721,-0.024590
Qty_3766,77.0,85.0,72.0,0.103896,-0.064935
Qty_16019,149.0,162.0,144.0,0.087248,-0.033557


In [42]:
elasticity_dict.keys()

dict_keys([202, 1195, 1230, 2531, 2589, 3726, 3732, 3743, 3766, 16019])

In [25]:
item_price_increase = mean_prices.copy()

In [35]:
decrease_quantities_dict

{'Qty_202': array([2097.], dtype=float32),
 'Qty_1195': array([125.], dtype=float32),
 'Qty_1230': array([9.], dtype=float32),
 'Qty_2531': array([256.], dtype=float32),
 'Qty_2589': array([128.], dtype=float32),
 'Qty_3726': array([61.], dtype=float32),
 'Qty_3732': array([121.], dtype=float32),
 'Qty_3743': array([122.], dtype=float32),
 'Qty_3766': array([77.], dtype=float32),
 'Qty_16019': array([149.], dtype=float32)}

In [27]:
item_price_increase

,Price_1195,Price_1230,Price_16019,Price_202,Price_2531,Price_2589,Price_3726,Price_3732,Price_3743,Price_3766,Qty_202
0,3.4,3.5,13.55,2.1,6.2,11.65,10.1,11.8,11.25,9.75,4000.0


In [28]:
mean_prices

,Price_1195,Price_1230,Price_16019,Price_202,Price_2531,Price_2589,Price_3726,Price_3732,Price_3743,Price_3766
0,3.4,3.5,13.55,2.1,6.2,11.65,10.1,11.8,11.25,9.75
